In [1]:
from enum import Enum
# Nên mở rộng class thay vì thay đổi class gốc

In [103]:
class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

class Size(Enum):
    SMALL = 1
    MEDIUM = 2
    LARGE = 3
class Product:
    def __init__(self, name, color, size):
        self.name = name
        self.color = color
        self.size = size


## OCP = Open for extension, closed for modification
class Product_Filter:
    def filter_by_color(self, products, color):
        for p in products:
            if p.color == color: yield p

    def filter_by_size(self, products, size):
        for p in products:
            if p.size == size: yield p

    def filter_by_size_and_color(self, products, size, color):
        for p in products:
            if p.color == color and p.size == size:
                yield p

class Specification:
    def is_satisfied(self, item):
        pass
    def __and__(self, other):
        return AndSpecification(self, other)
        
class Filter:
    def filter(self, items, spec):
        pass

class ColorSpecification(Specification):
    def __init__(self, color):
        self.color = color

    def is_satisfied(self, item):
        return item.color == self.color

class SizeSpecification(Specification):
    def __init__(self, size):
        self.size = size
    def is_satisfied(self, item):
        return item.size == self.size


class AndSpecification(Specification):
    def __init__(self, *args):
        self.args = args
    def is_satisfied(self,item):
        return all(
            map(
                lambda spec: spec.is_satisfied(item), self.args
            )
        )

class BetterFilter(Filter):
    def filter(self, items, spec):
        for item in items:
            if spec.is_satisfied(item):
                yield item

In [104]:
apple = Product('Apple',Color.GREEN, Size.SMALL)
tree = Product('Tree',Color.GREEN, Size.LARGE)
house = Product('House',Color.BLUE, Size.LARGE)

products = [apple, tree, house]

In [105]:
pf =  Product_Filter()
print('Green products (old):')
for p in pf.filter_by_color(products, Color.GREEN):
    print(f'- {p.name} is green')

Green products (old):
- Apple is green
- Tree is green


In [106]:
bf = BetterFilter()
print('Green products (new):')
green = ColorSpecification(Color.GREEN)
for p in bf.filter(products,green):
    print(f'- {p.name} is green')

Green products (new):
- Apple is green
- Tree is green


In [111]:
bf = BetterFilter()
large = SizeSpecification(Size.LARGE)
for p in bf.filter(products,large):
    print(f'- {p.name} is large')
    
 

- Tree is large
- House is large


In [110]:
large.is_satisfied(p)

True

In [112]:
print('Large Blue Items: ')
large_blue = AndSpecification(large, ColorSpecification(Color.BLUE))
for p in bf.filter(products, large_blue):
    print(f' - {p.name} is large and blue')
    

Large Blue Items: 
 - House is large and blue


In [113]:
for i in products:
    print(large_blue.is_satisfied(i))

False
False
True


In [114]:
large_blue = large&ColorSpecification(Color.BLUE)
for p in bf.filter(products, large_blue):
    print(f' - {p.name} is large and blue')

 - House is large and blue
